In [292]:
import pandas as pd
import os
import numpy as np
import ast

In [294]:
clean_data = pd.read_csv("tracks_filled_nulls.csv",index_col=[0],header=[0,1])

In [296]:
clean_data

album                                                        \
         comments         date_created date_released engineer favorites   
track_id                                                                  
2               0  2008-11-26 01:44:45    2009-01-05      NaN         4   
3               0  2008-11-26 01:44:45    2009-01-05      NaN         4   
5               0  2008-11-26 01:44:45    2009-01-05      NaN         4   
10              0  2008-11-26 01:45:08    2008-02-06      NaN         4   
20              0  2008-11-26 01:45:05    2009-01-06      NaN         2   
...           ...                  ...           ...      ...       ...   
155316          0  2017-03-30 15:20:35    2017-02-17      NaN         0   
155317          0  2017-03-30 15:20:35    2017-02-17      NaN         0   
155318          0  2017-03-30 15:20:35    2017-02-17      NaN         0   
155319          0  2017-03-30 15:20:35    2017-02-17      NaN         0   
155320          0  2017-03-26 16:22:18    2017-03-26      NaN         1   

                                                                            \
             id                                        information listens   
track_id                                                                     
2             1                                            <p></p>    6073   
3             1                                            <p></p>    6073   
5             1                                            <p></p>    6073   
10            6                                                NaN   47632   
20            4          <p> "spiritual songs" from Nicky Cook</p>    2710   
...         ...                                                ...     ...   
155316    22940  <p>A live performance at Monty Hall on Feb 17,...    1506   
155317    22940  <p>A live performance at Monty Hall on Feb 17,...    1506   
155318    22940  <p>A live performance at Monty Hall on Feb 17,...    1506   
155319    22940  <p>A live performance at Monty Hall on Feb 17,...    1506   
155320    22906                                                NaN    7481   

                                                                          ...  \
             producer                                               tags  ...   
track_id                                                                  ...   
2                 NaN                                                 []  ...   
3                 NaN                                                 []  ...   
5                 NaN                                                 []  ...   
10                NaN                                                 []  ...   
20                NaN                                                 []  ...   
...               ...                                                ...  ...   
155316    Monty Hall                                                  []  ...   
155317    Monty Hall                                                  []  ...   
155318    Monty Hall                                                  []  ...   
155319    Monty Hall                                                  []  ...   
155320            NaN  ['ballad', 'epic', 'rockabilly', 'curse', 'hex...  ...   

               track                         \
         information interest language_code   
track_id                                      
2                NaN     4656            en   
3                NaN     1470            en   
5                NaN     1933            en   
10               NaN    54881            en   
20               NaN      978            en   
...              ...      ...           ...   
155316           NaN      122           NaN   
155317           NaN      194           NaN   
155318           NaN      214           NaN   
155319           NaN      336           NaN   
155320           NaN      972           NaN   

                                                                          

In [298]:
def load(filepath):

    filename = os.path.basename(filepath)

    if 'features' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'echonest' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'genres' in filename:
        return pd.read_csv(filepath, index_col=0)

    if 'tracks' in filename:
        tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])

        COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
                   ('track', 'genres'), ('track', 'genres_all')]
        for column in COLUMNS:
            tracks[column] = tracks[column].map(ast.literal_eval)

        COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),
                   ('album', 'date_created'), ('album', 'date_released'),
                   ('artist', 'date_created'), ('artist', 'active_year_begin'),
                   ('artist', 'active_year_end')]
        for column in COLUMNS:
            tracks[column] = pd.to_datetime(tracks[column])

        SUBSETS = ('small', 'medium', 'large')
        try:
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                    'category', categories=SUBSETS, ordered=True)
        except (ValueError, TypeError):
            # the categories and ordered arguments were removed in pandas 0.25
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                     pd.CategoricalDtype(categories=SUBSETS, ordered=True))

        COLUMNS = [('track', 'genre_top'), ('track', 'license'),
                   ('album', 'type'), ('album', 'information'),
                   ('artist', 'bio')]
        for column in COLUMNS:
            tracks[column] = tracks[column].astype('category')

        return tracks


In [300]:
features = load("../../Genre Prediction/Data/Original Data/fma_metadata/features.csv")

In [411]:
genres = load("../../Genre Prediction/Data/Original Data/fma_metadata/genres.csv")

In [427]:
sortedG = genres.sort_values(by="#tracks",ascending = False)

In [451]:
sortedG.loc[[12,15,21,10,17]]

,#tracks,parent,title,top_level
genre_id,,,,
12,32923,0,Rock,12
15,34413,0,Electronic,15
21,8389,0,Hip-Hop,21
10,13845,0,Pop,10
17,12706,0,Folk,17


In [301]:
tracks = clean_data['track']

In [302]:
track_genre = tracks["genre_top"]

In [303]:
allCols = pd.MultiIndex.from_tuples([("track_id",'',''),("genre_top",'','')]+features.columns.to_list())

In [304]:
features = features.droplevel(0,axis=1).droplevel(0,axis=1)

In [305]:
track_genre_feature = pd.merge(track_genre.reset_index(),features.reset_index(),on="track_id",how="inner")

In [306]:
track_genre_feature.columns = allCols

In [307]:
genre_top_clean = pd.read_csv("genre_top_clean.csv")

In [308]:
genre_top_clean

,track_id,genre_top
0,2,21
1,3,21
2,5,21
3,10,10
4,134,21
...,...,...
49593,155315,12
49594,155316,12
49595,155317,12
49596,155318,12


In [309]:
track_genre_feature["genre_top"] = genre_top_clean["genre_top"]

In [378]:
track_genre_feature = track_genre_feature.dropna()

In [380]:
selected_genre = track_genre_feature[track_genre_feature["genre_top"].isin([38,15,12,1235,10,17,25,1,21,32,107,76,41,27,18,42,66])]

In [458]:
selected_genre.to_csv("selected_genre.csv")

In [382]:
X = selected_genre.drop(["track_id","genre_top"],axis=1)
y = selected_genre["genre_top"]

/var/folders/gv/571nb6gs4h58lv0gw0__gy280000gn/T/ipykernel_10936/2117871889.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X = selected_genre.drop(["track_id","genre_top"],axis=1)


In [384]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier

In [386]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [388]:
# from imblearn.under_sampling import RandomUnderSampler

# rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
# X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

In [390]:
model = RandomForestClassifier(n_estimators=100)

In [392]:
model.fit(X_train,y_train)

RandomForestClassifier()

In [393]:
y_pred = model.predict(X_test)

In [394]:
y_pred = pd.DataFrame(y_pred)

In [395]:
y_pred

,0
0,15.0
1,12.0
2,12.0
3,12.0
4,12.0
...,...
8981,12.0
8982,12.0
8983,12.0
8984,12.0


In [396]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred,average='micro')
recall = recall_score(y_test, y_pred,average='micro')
f1 = f1_score(y_test, y_pred,average='micro')
confusion = confusion_matrix(y_test, y_pred)
mean_squared_error = mean_squared_error(y_test, y_pred)
r2_score = r2_score(y_test, y_pred)


# Print metricsµ
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print(f"Confusion Matrix:\n{confusion}")
print(f"Mean Squared Error: {mean_squared_error}")
print(f"R2 Score: {r2_score}")

Accuracy: 0.33140440685510797
Precision: 0.33140440685510797
Recall: 0.33140440685510797
F1-score: 0.33140440685510797
Confusion Matrix:
[[   5  343   43    1    1   82    0]
 [   4 2272  187    0    5  392    3]
 [   3 1365  218    0    4  304    4]
 [   2  439   34    4    1   80    0]
 [   1  479   64    0   11  145    0]
 [   2 1463  162    3    5  463    0]
 [   1  290   36    1    1   58    5]]
Mean Squared Error: 65330.6744936568
R2 Score: -0.05884052827203767
